In [16]:
import pandas as pd
import numpy as np
import json

import warnings
warnings.filterwarnings('ignore')

In [17]:
df = pd.read_csv("../data/results.csv")

In [18]:
df.head()

,home_team,away_team,home_goals,away_goals,result,season
0,Sheffield United,Liverpool,1,1,D,2006-2007
1,Arsenal,Aston Villa,1,1,D,2006-2007
2,Everton,Watford,2,1,H,2006-2007
3,Newcastle United,Wigan Athletic,2,1,H,2006-2007
4,Portsmouth,Blackburn Rovers,3,0,H,2006-2007


### Odredi bodove i gol razliku klubova po utakmici: "home_points", "away_points", "home_goal_diff", "away_goal_diff"

In [19]:
df["home_points"] = df["result"].map({"H": 3, "D": 1, "A": 0})
df["away_points"] = df["result"].map({"H": 0, "D": 1, "A": 3})

In [20]:
df["home_goal_diff"] = df["home_goals"] - df["away_goals"]
df["away_goal_diff"] = df["away_goals"] - df["home_goals"]

In [21]:
df.head()

,home_team,away_team,home_goals,away_goals,result,season,home_points,away_points,home_goal_diff,away_goal_diff
0,Sheffield United,Liverpool,1,1,D,2006-2007,1,1,0,0
1,Arsenal,Aston Villa,1,1,D,2006-2007,1,1,0,0
2,Everton,Watford,2,1,H,2006-2007,3,0,1,-1
3,Newcastle United,Wigan Athletic,2,1,H,2006-2007,3,0,1,-1
4,Portsmouth,Blackburn Rovers,3,0,H,2006-2007,3,0,3,-3


### Podijeli dataframe po sezonama

In [22]:
unique_seasons = df["season"].unique()
unique_seasons

array(['2006-2007', '2007-2008', '2008-2009', '2009-2010', '2010-2011',
       '2011-2012', '2012-2013', '2013-2014', '2014-2015', '2015-2016',
       '2016-2017', '2017-2018'], dtype=object)

In [23]:
season_dict = {}

for season in unique_seasons:
    season_df = df[df["season"] == season]
    season_dict[season] = season_df

print(season_dict.keys())
print("Number of seasons: " + str(len(season_dict.keys())))

dict_keys(['2006-2007', '2007-2008', '2008-2009', '2009-2010', '2010-2011', '2011-2012', '2012-2013', '2013-2014', '2014-2015', '2015-2016', '2016-2017', '2017-2018'])
Number of seasons: 12


### Stvori novi dataframe s kolima - varijable "fixture_id", "team", "points", "goal_diff"

In [24]:
colors = ["#38B09D", "#F28E2B", "#2E77AE", "#E94877", "#4C566E",
          "#5E81AC", "#D08770", "#BF616A", "#A3BE8C", "#B48EAD",
          "#8FBCBB", "#EBCB8B", "#88C0D0", "#D8DEE9", "#A3BE8C",
          "#BF616A", "#81A1C1", "#ECEFF4", "#D8DEE9", "#A3BE8C"]

In [25]:
for season in season_dict.keys():
    df = pd.read_csv('../data/task3/points_season_'+season+'.csv')
    teams = np.array(df.team)
    
    result = []
    for club, color in zip(teams, colors):
        result.append({'team': club, 'color': color})

    a = np.array(season_dict[season]['away_team'])
    h = np.array(season_dict[season]['home_team'])
    h_p = np.array(season_dict[season]['home_points'])
    a_p = np.array(season_dict[season]['away_points'])

    for i in range(len(h)):
        team_h = h[i]
        team_a = a[i]

        for team_dict in result:
            if team_dict['team'] == team_h:
                if 'positions' in team_dict:
                    dictio = {}
                    current_points = team_dict['positions'][len(team_dict['positions'])-1]
                    dictio['points'] = str(int(current_points['points']) + h_p[i])
                    team_dict['positions'].append(dictio)
                else:
                    team_dict['positions'] = []
                    dictio = {}
                    dictio['points'] = str(h_p[i])
                    team_dict['positions'].append(dictio)
            
            if team_dict['team'] == team_a:
                if 'positions' in team_dict:
                    dictio = {}
                    current_points = team_dict['positions'][len(team_dict['positions'])-1]
                    dictio['points'] = str(int(current_points['points']) + a_p[i])
                    team_dict['positions'].append(dictio)
                else:
                    team_dict['positions'] = []
                    dictio = {}
                    dictio['points'] = str(a_p[i])
                    team_dict['positions'].append(dictio)

    # Save the list to a JSON file
    with open('../data/task5/point_per_game_' + season + '.json', 'w') as json_file:
        json.dump(result, json_file)

DONE

In [10]:
"""
season_with_fixtures = dict()

for season, season_df in season_dict.items():
    season_df = season_df.reset_index(drop=True)
    fixture_data = []
    fixture_id = 1
    for idx, row in season_df.iterrows():
        fixture_data.append({
            "fixture_id": fixture_id,
            "team": row["home_team"],
            "points": row["home_points"],
            "goal_diff": row["home_goal_diff"]
        })

        fixture_data.append({
            "fixture_id": fixture_id,
            "team": row["away_team"],
            "points": row["away_points"],
            "goal_diff": row["away_goal_diff"]
        })

        if idx % 10 == 9:
            fixture_id += 1

    season_with_fixtures[season] = pd.DataFrame(fixture_data)
"""

In [26]:
"""
print(season_with_fixtures.keys())
print("Number of seasons: " + str(len(season_with_fixtures.keys())))
"""

'\nprint(season_with_fixtures.keys())\nprint("Number of seasons: " + str(len(season_with_fixtures.keys())))\n'

In [27]:
"""
season_with_fixtures["2006-2007"].head(n=30)
"""

'\nseason_with_fixtures["2006-2007"].head(n=30)\n'

### Izračunaj rang po kolu

Sample

In [13]:
#sample_df = season_with_fixtures["2006-2007"]

In [14]:
#sample_df["team"] = sample_df.groupby("fixture_id")["team"].transform(lambda x: sorted(x))

In [28]:
#sample_df["cumulative_points"] = sample_df.groupby("team")["points"].cumsum()
#sample_df.head(n=60)

### Spremi po sezoni

In [ ]:
"""
for season, season_df in season_with_fixtures.items():
    season_df["team"] = season_df.groupby("fixture_id")["team"].transform(lambda x: sorted(x))
    season_df["cumulative_points"] = season_df.groupby("team")["points"].cumsum()
    season_csv_filename = f'season_{season}.csv'
    season_df.to_csv("../data/task5/" + season_csv_filename, index=False)
    print(f"Saved {season_csv_filename}")
"""